In [7]:
search_for = 98
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00434565544128418
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 212822781
type: [1, 1, 1, 1, 98] 98
cases of this type: 92236816
10000 35.562739204706865
20000 34.28088264844258
30000 42.710916986933434
40000 150.82146486850982
50000 132.56032550244052
60000 124.25523207255112
70000 121.73621985914966
80000 126.84661602718246
90000 126.22633964014666
100000 123.72878664948387
110000 121.90567590495355
120000 122.20612883784798
130000 123.47437698025568
140000 126.13956810675994
150000 120.43652575621502
160000 101.49751630213845
170000 115.54593520585514
180000 116.9157029349167
190000 118.8152793831756
200000 100.98277535611957
210000 120.48099043840153
220000 124.8624121718694
230000 130.93531654637823
240000 131.14930778905833
250000 131.24655365370594
260000 127.73785257144905
270000 124.43869065781641
280000 115.14176776431678
290000 126.07028669046775
300000 114.94672735921452
310000 12

3040000 106.00868932392005
3050000 116.63791620258284
3060000 108.10746811233693
3070000 120.45086289253383
3080000 103.56913592128046
3090000 119.23720208797857
3100000 106.33919253028255
3110000 102.7076680629127
3120000 106.17566207410312
3130000 116.18130086669578
3140000 106.48402681020691
3150000 92.25260299869824
3160000 102.43049076277781
3170000 106.71262197615772
3180000 98.39532559756815
3190000 97.42073983277422
3200000 107.92604924614373
3210000 115.71219653118257
3220000 111.9269159959698
3230000 103.24131823141472
3240000 109.50519832389087
3250000 107.75000985183034
3260000 102.96399565919822
3270000 123.53208677294015
3280000 71.02534262990837
3290000 17.592574202755166
3300000 16.043395427022993
3310000 18.170915157456655
3320000 12.868863117087256
3330000 114.16485478223329
3340000 138.21562756570714
3350000 105.96274626101435
3360000 103.87466597547629
3370000 103.1115650698299
3380000 116.04680831456184
3390000 100.35452384305817
3400000 102.994676452869
3410000 96

6130000 15.960135431304854
6140000 17.740709560974036
6150000 51.29892706067806
6160000 114.04462359857261
6170000 89.72932361383693
6180000 88.97941546407658
6190000 102.42937111330212
6200000 97.11214919717905
6210000 101.64658161499212
6220000 106.56349220170318
6230000 100.18405497319091
6240000 110.76854070499493
6250000 113.39852007210614
6260000 108.85226881150243
6270000 98.54430755414201
6280000 108.16331352363319
6290000 101.65438373069289
6300000 92.58091241647877
6310000 97.77472863556036
6320000 110.10200364706888
6330000 105.65291606850764
6340000 115.8339006248112
6350000 104.87985263913214
6360000 103.9420373102535
6370000 105.5361439699497
6380000 111.83641179917383
6390000 105.03790771606138
6400000 107.61186730311546
6410000 89.54120042739643
6420000 105.64410567853857
6430000 94.72607929459662
6440000 98.07710041307082
6450000 103.59154390259165
6460000 109.3738957497674
6470000 95.17493972409218
6480000 109.70825624604079
6490000 101.87251766000836
6500000 111.3262

9230000 110.6789757922604
9240000 112.76157598877316
9250000 105.59228276372801
9260000 103.10656521512227
9270000 97.59635753775474
9280000 114.66988898138534
9290000 108.86610772511233
9300000 121.49961927010364
9310000 91.36034538884346
9320000 37.4944824796836
9330000 35.81858497280115
9340000 37.32343883182737
9350000 37.56777583112358
9360000 88.63124728812853
9370000 112.33281187099111
9380000 94.0480438753355
9390000 84.39432427378158
9400000 100.54624818243066
9410000 121.95212012739472
9420000 118.09270118658829
9430000 116.3165497387779
9440000 111.77430286955531
9450000 112.51734097920428
9460000 91.70454842902117
9470000 106.24223213947795
9480000 102.33909628502771
9490000 123.90140359217123
9500000 26.70023038871016
9510000 35.24783400801301
9520000 34.5497259459025
9530000 31.66912394365524
9540000 68.63152610335902
9550000 129.1735494305326
9560000 102.62235531440109
9570000 88.42634814081482
9580000 98.79780282561252
9590000 98.97772287712345
9600000 101.5469048102933

12260000 108.37056817164859
12270000 103.75828000963838
12280000 99.53640847225495
12290000 106.4933875372123
12300000 107.46726918073922
12310000 109.70495409671688
12320000 98.98978475904318
12330000 114.12915607551923
12340000 118.74973349203847
12350000 38.54364975729091
12360000 31.422094217303734
12370000 33.07869447763898
12380000 30.191945525249793
12390000 42.704997110801514
12400000 119.05806898313206
12410000 109.53731546757007
12420000 100.841305969331
12430000 99.9088729705774
12440000 116.84169622021808
12450000 104.81333606150547
12460000 113.01760010203625
12470000 108.79600568589572
12480000 109.949821492343
12490000 103.8969559085141
12500000 111.97149312301445
12510000 103.35811545554625
12520000 94.52251347455923
12530000 100.21082061071107
12540000 110.83507570205023
12550000 117.51742162950488
12560000 98.35461685563428
12570000 104.26587476959806
12580000 93.5989093299154
12590000 93.07937955813688
12600000 84.9147058509762
12610000 94.36356528637246
12620000 97.

...

200390000 3.4305464749285974
200400000 3.8876333987715843
200410000 3.524443243357049
200420000 1.4666578382876516
200430000 0.9391025497306984
200440000 2.4740220897245937
200450000 3.900260758291099
200460000 3.989685462939521
200470000 3.3769800530780287
200480000 2.5657580539731453
200490000 1.9483121391571163
200500000 2.1903356968013314
200510000 3.7372744141683376
200520000 3.972599319942872
200530000 3.724122567149136
200540000 3.8398406865503456
200550000 4.287463312234223
200560000 3.9609599706153937
200570000 4.056836555193371
200580000 3.646888205428123
200590000 1.9857397033414244
200600000 1.668665981359197
200610000 3.1726456667073766
200620000 3.769853700606618
200630000 3.9592899744041694
200640000 3.8080465178431075
200650000 3.0839677731823327
200660000 1.0513334744667213
200670000 0.9203638999165893
200680000 4.077731208654689
200690000 3.4422638103996253
200700000 3.815749641459505
200710000 3.82899360345649
200720000 2.7276638267098527
200730000 1.8313306489759882

203240000 1.1720958628694218
203250000 0.90989787880824
203260000 0.964052305961238
203270000 1.2450550038246315
203280000 0.9828563744594456
203290000 0.8792466070432728
203300000 1.1309673133211864
203310000 1.299386200900356
203320000 1.0437551527363924
203330000 1.0268713958011468
203340000 1.7654151338550848
203350000 1.5627452017009458
203360000 1.5161919526553815
203370000 1.7217600754775406
203380000 1.5160569217752946
203390000 1.4713051044309802
203400000 1.7293828370837372
203410000 1.3959383506573437
203420000 1.1600223223881325
203430000 1.2279120999314388
203440000 1.2318951943984762
203450000 1.0055534981414147
203460000 0.964614147571683
203470000 1.2407617139109044
203480000 1.165503003444771
203490000 0.876663715506335
203500000 1.0742693399520253
203510000 1.0918728014618424
203520000 0.916565132182022
203530000 0.9851656854190495
203540000 1.1383231515077419
203550000 1.1042257334749699
203560000 1.0208166930831273
203570000 1.4136270405665903
203580000 1.5197270243

best so far: 0
type: [1, 7, 7, 2, 1] 98
cases of this type: 2744
205830000 0.4635455321496526
best so far: 0
type: [1, 7, 14, 1, 1] 98
cases of this type: 1372
best so far: 0
type: [1, 14, 1, 1, 7] 98
cases of this type: 33614
205840000 0.5262732004915212
205850000 0.46908627501064537
205860000 0.45668366990027826
best so far: 0
type: [1, 14, 1, 7, 1] 98
cases of this type: 4802
205870000 0.6228898403786554
best so far: 0
type: [1, 14, 7, 1, 1] 98
cases of this type: 686
best so far: 0
type: [1, 49, 1, 1, 2] 98
cases of this type: 784
best so far: 0
type: [1, 49, 1, 2, 1] 98
cases of this type: 392
best so far: 0
type: [1, 49, 2, 1, 1] 98
cases of this type: 196
best so far: 0
type: [1, 98, 1, 1, 1] 98
cases of this type: 98
best so far: 0
type: [2, 1, 1, 1, 49] 98
cases of this type: 5764801
205880000 0.7894010446468592
205890000 2.4362149956356287
205900000 2.2659872766560847
205910000 2.552197906262444
205920000 2.0593228552471
205930000 1.7331828589080902
205940000 0.95614524184811

208450000 1.3739280391151374
208460000 1.3606850822161833
208470000 0.5622879293457469
208480000 0.3309231123672989
208490000 0.334342866479675
208500000 0.33411085766386983
208510000 0.31868868514419924
208520000 0.31431112320193977
208530000 0.32637398171788456
208540000 0.31966990462483963
208550000 0.3078247043758962
208560000 0.27959923936446507
208570000 0.3087422699985968
208580000 0.31331890682196617
208590000 0.2753518760640025
208600000 0.30776694792795845
208610000 0.2560273369191686
208620000 0.33252901584651073
208630000 0.3281186464026372
208640000 0.3278616185986598
208650000 0.3236928802017967
208660000 0.30359538795310925
208670000 0.2950406298794018
208680000 0.28809158608001467
208690000 0.30816505768320956
208700000 0.2848765593477885
208710000 0.3067888988766472
208720000 0.28687931371556386
208730000 0.3071107502350211
208740000 0.30491670198164383
208750000 0.2988885331008302
208760000 0.3196820709862775
208770000 0.2838924018750787
208780000 0.24793165901705952


211270000 0.4665683663193716
211280000 0.27912626036898297
211290000 0.32814562760931254
211300000 0.3153503881340027
211310000 0.4483612237254778
211320000 0.4703834064903458
211330000 0.3886358678091632
211340000 0.377494264213211
211350000 0.09774806108244259
211360000 0.45364296189865144
211370000 0.4370865025309125
211380000 0.42456551974952217
211390000 0.4264126645516753
211400000 0.2481972199418346
211410000 0.26963161109882594
211420000 0.37858235524347095
211430000 0.418426997046729
211440000 0.4545417811971505
211450000 0.42077164720906146
211460000 0.3151664475893113
211470000 0.20646409391194584
211480000 0.3647603444556991
211490000 0.45286225753391457
211500000 0.42420637441406644
211510000 0.3955853111520277
211520000 0.2406109575920304
211530000 0.30826391715220613
211540000 0.4257493226336903
211550000 0.4452523557885024
211560000 0.4463441703901291
211570000 0.3739998834509651
211580000 0.22690615763505304
211590000 0.2681969516467849
211600000 0.402835917524351
2116